# DDSM Mask RCNN training


In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.getcwd()
print(ROOT_DIR)
# if ROOT_DIR.endswith("samples/nucleus"):
if ROOT_DIR.endswith("mammography"):
    # Go up one level to the repo root
    ROOT_DIR = os.path.dirname(ROOT_DIR)
    print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

import mammo

%matplotlib inline 

C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN\mammography
C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN


C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

## Configurations

In [2]:
# Dataset directory
# DATASET_DIR = os.path.join(ROOT_DIR, "datasets/nucleus")
DATASET_DIR = os.path.join(ROOT_DIR, "datasets/mammo")



In [3]:
# Load dataset
dataset_train = mammo.MammoDataset()
dataset_train.load_mammo(DATASET_DIR, subset="train")

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    

# Load validation dataset
dataset_val = mammo.MammoDataset()
dataset_val.load_mammo(DATASET_DIR, "val")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))




train
Image Count: 984
Class Count: 2
  0. BG                                                
  1. mass                                              
val
Images: 245
Classes: ['BG', 'mass']


## Notebook Preferences

In [4]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2500
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Con

Epoch 11/60
984/984 [==============================] - 1173s 1s/step - loss: 0.7815 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.1146 - mrcnn_class_loss: 0.1262 - mrcnn_bbox_loss: 0.1936 - mrcnn_mask_loss: 0.3443 - val_loss: 1.1722 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3903 - val_mrcnn_class_loss: 0.1228 - val_mrcnn_bbox_loss: 0.2902 - val_mrcnn_mask_loss: 0.3658
Epoch 12/60
984/984 [==============================] - 1177s 1s/step - loss: 0.7264 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0962 - mrcnn_class_loss: 0.1249 - mrcnn_bbox_loss: 0.1706 - mrcnn_mask_loss: 0.3321 - val_loss: 1.1936 - val_rpn_class_loss: 0.0027 - val_rpn_bbox_loss: 0.3595 - val_mrcnn_class_loss: 0.1290 - val_mrcnn_bbox_loss: 0.3180 - val_mrcnn_mask_loss: 0.3843
Epoch 13/60
984/984 [==============================] - 1178s 1s/step - loss: 0.6738 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0799 - mrcnn_class_loss: 0.1157 - mrcnn_bbox_loss: 0.1535 - mrcnn_mask_loss: 0.3223 - val_loss: 1.3115 - val_rpn_c

Epoch 33/60
984/984 [==============================] - 1178s 1s/step - loss: 0.2788 - rpn_class_loss: 9.7027e-04 - rpn_bbox_loss: 0.0257 - mrcnn_class_loss: 0.0380 - mrcnn_bbox_loss: 0.0361 - mrcnn_mask_loss: 0.1780 - val_loss: 2.0119 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.3080 - val_mrcnn_class_loss: 0.6276 - val_mrcnn_bbox_loss: 0.3551 - val_mrcnn_mask_loss: 0.7187
Epoch 34/60
984/984 [==============================] - 1176s 1s/step - loss: 0.2730 - rpn_class_loss: 9.1119e-04 - rpn_bbox_loss: 0.0252 - mrcnn_class_loss: 0.0376 - mrcnn_bbox_loss: 0.0357 - mrcnn_mask_loss: 0.1735 - val_loss: 2.1726 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3216 - val_mrcnn_class_loss: 0.7483 - val_mrcnn_bbox_loss: 0.3666 - val_mrcnn_mask_loss: 0.7331
Epoch 35/60
984/984 [==============================] - 1177s 1s/step - loss: 0.2653 - rpn_class_loss: 9.1526e-04 - rpn_bbox_loss: 0.0260 - mrcnn_class_loss: 0.0347 - mrcnn_bbox_loss: 0.0343 - mrcnn_mask_loss: 0.1694 - val_loss: 2.3798

984/984 [==============================] - 1182s 1s/step - loss: 0.1798 - rpn_class_loss: 5.4522e-04 - rpn_bbox_loss: 0.0143 - mrcnn_class_loss: 0.0221 - mrcnn_bbox_loss: 0.0186 - mrcnn_mask_loss: 0.1241 - val_loss: 2.4589 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3091 - val_mrcnn_class_loss: 1.0551 - val_mrcnn_bbox_loss: 0.3371 - val_mrcnn_mask_loss: 0.7546
Epoch 55/60
984/984 [==============================] - 1180s 1s/step - loss: 0.1762 - rpn_class_loss: 5.3329e-04 - rpn_bbox_loss: 0.0150 - mrcnn_class_loss: 0.0209 - mrcnn_bbox_loss: 0.0180 - mrcnn_mask_loss: 0.1218 - val_loss: 2.8937 - val_rpn_class_loss: 0.0037 - val_rpn_bbox_loss: 0.3054 - val_mrcnn_class_loss: 1.3224 - val_mrcnn_bbox_loss: 0.3503 - val_mrcnn_mask_loss: 0.9119
Epoch 56/60
984/984 [==============================] - 1180s 1s/step - loss: 0.1755 - rpn_class_loss: 5.2484e-04 - rpn_bbox_loss: 0.0148 - mrcnn_class_loss: 0.0217 - mrcnn_bbox_loss: 0.0180 - mrcnn_mask_loss: 0.1204 - val_loss: 2.4129 - val_rpn_c

## Training #2 
### With proper bounding box and 60 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

Epoch 11/60
984/984 [==============================] - 809s 823ms/step - loss: 0.8031 - rpn_class_loss: 0.0030 - rpn_bbox_loss: 0.1218 - mrcnn_class_loss: 0.1195 - mrcnn_bbox_loss: 0.2000 - mrcnn_mask_loss: 0.3588 - val_loss: 1.1603 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.2916 - val_mrcnn_class_loss: 0.1418 - val_mrcnn_bbox_loss: 0.3378 - val_mrcnn_mask_loss: 0.3862
Epoch 12/60
984/984 [==============================] - 810s 824ms/step - loss: 0.7393 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0982 - mrcnn_class_loss: 0.1174 - mrcnn_bbox_loss: 0.1757 - mrcnn_mask_loss: 0.3452 - val_loss: 1.1646 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.3261 - val_mrcnn_class_loss: 0.1456 - val_mrcnn_bbox_loss: 0.3173 - val_mrcnn_mask_loss: 0.3727
Epoch 13/60
984/984 [==============================] - 807s 820ms/step - loss: 0.7080 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0878 - mrcnn_class_loss: 0.1193 - mrcnn_bbox_loss: 0.1592 - mrcnn_mask_loss: 0.3391 - val_loss: 1.2166 - val

984/984 [==============================] - 831s 845ms/step - loss: 0.1840 - rpn_class_loss: 6.5612e-04 - rpn_bbox_loss: 0.0159 - mrcnn_class_loss: 0.0209 - mrcnn_bbox_loss: 0.0189 - mrcnn_mask_loss: 0.1277 - val_loss: 2.3100 - val_rpn_class_loss: 0.0033 - val_rpn_bbox_loss: 0.3501 - val_mrcnn_class_loss: 0.8233 - val_mrcnn_bbox_loss: 0.3451 - val_mrcnn_mask_loss: 0.7882
Epoch 54/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1850 - rpn_class_loss: 6.8979e-04 - rpn_bbox_loss: 0.0157 - mrcnn_class_loss: 0.0220 - mrcnn_bbox_loss: 0.0192 - mrcnn_mask_loss: 0.1274 - val_loss: 2.1762 - val_rpn_class_loss: 0.0034 - val_rpn_bbox_loss: 0.3517 - val_mrcnn_class_loss: 0.6564 - val_mrcnn_bbox_loss: 0.3617 - val_mrcnn_mask_loss: 0.8030
Epoch 55/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1807 - rpn_class_loss: 6.4021e-04 - rpn_bbox_loss: 0.0145 - mrcnn_class_loss: 0.0232 - mrcnn_bbox_loss: 0.0187 - mrcnn_mask_loss: 0.1236 - val_loss: 2.1490 - val

## Training #1 
### With proper bounding box and 40 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=40, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Conv2D)
bn4o_branch2a          (BatchNorm)
res4o_branch2b         (Co

Epoch 11/40
984/984 [==============================] - 1350s 1s/step - loss: 0.6850 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.1114 - mrcnn_class_loss: 0.0289 - mrcnn_bbox_loss: 0.1996 - mrcnn_mask_loss: 0.3425 - val_loss: 1.1400 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3213 - val_mrcnn_class_loss: 0.0514 - val_mrcnn_bbox_loss: 0.3276 - val_mrcnn_mask_loss: 0.4367
Epoch 12/40
984/984 [==============================] - 1358s 1s/step - loss: 0.6486 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0993 - mrcnn_class_loss: 0.0269 - mrcnn_bbox_loss: 0.1840 - mrcnn_mask_loss: 0.3360 - val_loss: 1.0837 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.4048 - val_mrcnn_class_loss: 0.0297 - val_mrcnn_bbox_loss: 0.2958 - val_mrcnn_mask_loss: 0.3509
Epoch 13/40
984/984 [==============================] - 1358s 1s/step - loss: 0.5991 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0865 - mrcnn_class_loss: 0.0244 - mrcnn_bbox_loss: 0.1646 - mrcnn_mask_loss: 0.3212 - val_loss: 1.0945 - val_rpn_c

Epoch 33/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2596 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0066 - mrcnn_bbox_loss: 0.0409 - mrcnn_mask_loss: 0.1829 - val_loss: 1.3726 - val_rpn_class_loss: 0.0023 - val_rpn_bbox_loss: 0.3322 - val_mrcnn_class_loss: 0.0812 - val_mrcnn_bbox_loss: 0.3456 - val_mrcnn_mask_loss: 0.6113
Epoch 34/40
984/984 [==============================] - 1361s 1s/step - loss: 0.2577 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0067 - mrcnn_bbox_loss: 0.0406 - mrcnn_mask_loss: 0.1811 - val_loss: 1.7583 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3568 - val_mrcnn_class_loss: 0.1285 - val_mrcnn_bbox_loss: 0.3535 - val_mrcnn_mask_loss: 0.9164
Epoch 35/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2454 - rpn_class_loss: 9.7563e-04 - rpn_bbox_loss: 0.0250 - mrcnn_class_loss: 0.0055 - mrcnn_bbox_loss: 0.0370 - mrcnn_mask_loss: 0.1769 - val_loss: 1.5053 - val_r

## Attempt 4 - Completed on 13/07/2018
### Epochs trained: 75/100

In [5]:
# Configurations
# config = nucleus.MammoConfig()
# Use configuation from nucleus.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(nucleus.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     BACKBONE = "resnet50"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=100, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
Epoch 1/100
1348/1348 [

Epoch 20/100
1348/1348 [==============================] - 1637s 1s/step - loss: 0.0233 - rpn_class_loss: 3.4654e-04 - rpn_bbox_loss: 0.0057 - mrcnn_class_loss: 9.7076e-04 - mrcnn_bbox_loss: 0.0016 - mrcnn_mask_loss: 0.0146 - val_loss: 0.0430 - val_rpn_class_loss: 2.7081e-04 - val_rpn_bbox_loss: 0.0228 - val_mrcnn_class_loss: 0.0015 - val_mrcnn_bbox_loss: 0.0021 - val_mrcnn_mask_loss: 0.0163
Epoch 21/100
1348/1348 [==============================] - 1633s 1s/step - loss: 0.0224 - rpn_class_loss: 3.2356e-04 - rpn_bbox_loss: 0.0054 - mrcnn_class_loss: 9.7481e-04 - mrcnn_bbox_loss: 0.0015 - mrcnn_mask_loss: 0.0142 - val_loss: 0.0512 - val_rpn_class_loss: 3.5787e-04 - val_rpn_bbox_loss: 0.0322 - val_mrcnn_class_loss: 8.9098e-04 - val_mrcnn_bbox_loss: 0.0021 - val_mrcnn_mask_loss: 0.0157
Epoch 22/100
1348/1348 [==============================] - 1669s 1s/step - loss: 0.0228 - rpn_class_loss: 3.5126e-04 - rpn_bbox_loss: 0.0063 - mrcnn_class_loss: 0.0011 - mrcnn_bbox_loss: 0.0015 - mrcnn_mask_lo

Epoch 41/100
1348/1348 [==============================] - 1643s 1s/step - loss: 0.0116 - rpn_class_loss: 2.6739e-04 - rpn_bbox_loss: 0.0029 - mrcnn_class_loss: 5.6214e-04 - mrcnn_bbox_loss: 7.9794e-04 - mrcnn_mask_loss: 0.0071 - val_loss: 0.0581 - val_rpn_class_loss: 3.1746e-04 - val_rpn_bbox_loss: 0.0346 - val_mrcnn_class_loss: 5.3967e-04 - val_mrcnn_bbox_loss: 0.0017 - val_mrcnn_mask_loss: 0.0210
Epoch 42/100
1348/1348 [==============================] - 1653s 1s/step - loss: 0.0111 - rpn_class_loss: 1.7736e-04 - rpn_bbox_loss: 0.0029 - mrcnn_class_loss: 5.5940e-04 - mrcnn_bbox_loss: 7.3684e-04 - mrcnn_mask_loss: 0.0068 - val_loss: 0.0491 - val_rpn_class_loss: 2.1855e-04 - val_rpn_bbox_loss: 0.0247 - val_mrcnn_class_loss: 3.2725e-04 - val_mrcnn_bbox_loss: 0.0013 - val_mrcnn_mask_loss: 0.0225
Epoch 43/100
1348/1348 [==============================] - 1657s 1s/step - loss: 0.0112 - rpn_class_loss: 2.2969e-04 - rpn_bbox_loss: 0.0033 - mrcnn_class_loss: 5.0832e-04 - mrcnn_bbox_loss: 7.6171

KeyboardInterrupt: 

## Attempt 3 @ 09/07/2018

In [ ]:
# Configurations
config = nucleus.MammoConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=100, layers='all')


## Attempt 2

In [6]:
# Configurations
config = nucleus.MammoConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=100, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCE

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



res5c_branch2c         (Conv2D)
bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_dec

Epoch 20/100
645/645 [==============================] - 724s 1s/step - loss: 0.0314 - rpn_class_loss: 6.5898e-04 - rpn_bbox_loss: 0.0099 - mrcnn_class_loss: 0.0036 - mrcnn_bbox_loss: 0.0016 - mrcnn_mask_loss: 0.0156 - val_loss: 0.3077 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.2786 - val_mrcnn_class_loss: 0.0071 - val_mrcnn_bbox_loss: 0.0054 - val_mrcnn_mask_loss: 0.0137
Epoch 21/100
645/645 [==============================] - 724s 1s/step - loss: 0.0300 - rpn_class_loss: 4.8396e-04 - rpn_bbox_loss: 0.0085 - mrcnn_class_loss: 0.0038 - mrcnn_bbox_loss: 0.0016 - mrcnn_mask_loss: 0.0156 - val_loss: 0.2990 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.2714 - val_mrcnn_class_loss: 0.0051 - val_mrcnn_bbox_loss: 0.0052 - val_mrcnn_mask_loss: 0.0144
Epoch 22/100
645/645 [==============================] - 718s 1s/step - loss: 0.0285 - rpn_class_loss: 5.7595e-04 - rpn_bbox_loss: 0.0087 - mrcnn_class_loss: 0.0036 - mrcnn_bbox_loss: 0.0014 - mrcnn_mask_loss: 0.0143 - val_loss: 0.2924

645/645 [==============================] - 735s 1s/step - loss: 0.0131 - rpn_class_loss: 3.5251e-04 - rpn_bbox_loss: 0.0047 - mrcnn_class_loss: 0.0031 - mrcnn_bbox_loss: 7.2329e-04 - mrcnn_mask_loss: 0.0043 - val_loss: 0.2404 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.2094 - val_mrcnn_class_loss: 0.0044 - val_mrcnn_bbox_loss: 0.0023 - val_mrcnn_mask_loss: 0.0213
Epoch 63/100
645/645 [==============================] - 735s 1s/step - loss: 0.0104 - rpn_class_loss: 1.6936e-04 - rpn_bbox_loss: 0.0026 - mrcnn_class_loss: 0.0029 - mrcnn_bbox_loss: 6.2032e-04 - mrcnn_mask_loss: 0.0041 - val_loss: 0.2775 - val_rpn_class_loss: 0.0033 - val_rpn_bbox_loss: 0.2479 - val_mrcnn_class_loss: 0.0022 - val_mrcnn_bbox_loss: 0.0020 - val_mrcnn_mask_loss: 0.0223
Epoch 64/100
645/645 [==============================] - 735s 1s/step - loss: 0.0119 - rpn_class_loss: 2.7936e-04 - rpn_bbox_loss: 0.0037 - mrcnn_class_loss: 0.0031 - mrcnn_bbox_loss: 7.7335e-04 - mrcnn_mask_loss: 0.0041 - val_loss: 0.2776 

## Attempt 1

In [16]:
# Configurations
config = nucleus.MammoConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=40, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCE

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn5b_branch2c          (BatchNorm)
res5c_branch2a         (Conv2D)
bn5c_branch2a          (BatchNorm)
res5c_branch2b         (Conv2D)
bn5c_branch2b          (BatchNorm)
res5c_branch2c         (Conv2D)
bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_clas

649/649 [==============================] - 487s 750ms/step - loss: 0.0766 - rpn_class_loss: 0.0025 - rpn_bbox_loss: 0.0436 - mrcnn_class_loss: 0.0092 - mrcnn_bbox_loss: 0.0020 - mrcnn_mask_loss: 0.0192 - val_loss: 0.4176 - val_rpn_class_loss: 0.0109 - val_rpn_bbox_loss: 0.3614 - val_mrcnn_class_loss: 0.0170 - val_mrcnn_bbox_loss: 0.0043 - val_mrcnn_mask_loss: 0.0240
